# Задание

Сделать классификацию данных fakenews ([Материалы](https://github.com/netology-ds-team/nlp-homeworks/tree/main/7_Classification_in_AOT/Materials))
3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn   
и выше 0.52 для методов на pytorch.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Используя ноутбук занятия (также размещен в папке Materials) и данные fakenews,
1. 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn
2. и выше 0.52 для методов на pytorch.

In [ ]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2023-09-29 12:10:38--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.04s   

2023-09-29 12:10:38 (29.0 MB/s) - ‘Constraint_Train.csv’ saved [1253562/1253562]



In [ ]:
! pip install gensim==3.8.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 52.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-linux_x86_64.whl size=26528003 sha256=970128551566a167f96c42f92c62d6f112129585c213a05dcb02f7d6f0813f0b
  Stored in directory: /root/.cache/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1


In [ ]:
print(gensim.__version__)

3.8.3


In [ ]:
import gensim
from gensim.models import Word2Vec

### Загрузка данных

In [ ]:
df = pd.read_csv('Constraint_Train.csv')

In [ ]:
df

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
...,...,...,...
6415,6416,A tiger tested positive for COVID-19 please st...,fake
6416,6417,???Autopsies prove that COVID-19 is??� a blood...,fake
6417,6418,_A post claims a COVID-19 vaccine has already ...,fake
6418,6419,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,fake


In [ ]:
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm
tqdm.pandas()
import nltk
from nltk.corpus import stopwords
import re

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### Предобработка

In [ ]:
# препроцессинг

mystopwords = stopwords.words('english') + ['http', 'https']
words = re.compile("[A-Za-z]+")

def words_only(text):
    return " ".join(words.findall(text))

def remove_word(text):
    return " ".join([token for token in text.split() if len(token) >= 3])

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

def lemmatize(text):
    lemmatizer = nltk.WordNetLemmatizer()
    try:
        return  " ".join([lemmatizer.lemmatize(w) for w in text.split()])
    except:
        return " "

def preprocess(text):
    return remove_stopwords(lemmatize(remove_word(words_only(text.lower()))))


In [ ]:
df.tweet = df.tweet.astype('str').progress_apply(preprocess)

  0%|          | 0/6420 [00:00<?, ?it/s]

In [ ]:
df.tweet[1]

'state reported death small rise last tuesday southern state reported death yasgrtt'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df.tweet)

In [ ]:
X.shape

(6420, 15904)

In [ ]:
df.label.value_counts()

real    3360
fake    3060
Name: label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
le.fit(df.label)
y = le.transform(df.label)

In [ ]:
len(y)

6420

### 1-й способ (sklearn)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
clf_tf_idf = Pipeline(
    [('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', LogisticRegression())]
)

NameError: ignored

In [ ]:
clf_tf_idf.fit(X_train, y_train)

In [ ]:
%%time

param_grid = {'max_iter': [1000, 2000, 3000, 5000, 10000],
              'epsilon' : [0.1, 0.2, 0.5, 0.01, 0.001],
              'learning_rate' : ['constant', 'optimal', 'invscaling', 'adaptive']

}
sgd = SGDClassifier()
grid = GridSearchCV(sgd, param_grid, cv=10, scoring='accuracy')

grid.fit(X_train,y_train)

CPU times: user 4.31 s, sys: 19 ms, total: 4.33 s
Wall time: 4.32 s


GridSearchCV(cv=10, estimator=SGDClassifier(),
             param_grid={'epsilon': [0.1, 0.2, 0.5, 0.01, 0.001],
                         'learning_rate': ['constant', 'optimal', 'invscaling',
                                           'adaptive'],
                         'max_iter': [1000, 2000, 3000, 5000, 10000]},
             scoring='accuracy')

In [ ]:
print(grid.best_score_)
print(grid.best_estimator_)

0.9360605406572061
SGDClassifier(epsilon=0.01, max_iter=3000)


In [ ]:
model = SGDClassifier(epsilon=0.01, max_iter=3000, random_state=42)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93      1038
           1       0.95      0.92      0.94      1081

    accuracy                           0.93      2119
   macro avg       0.93      0.94      0.93      2119
weighted avg       0.94      0.93      0.93      2119



In [ ]:
%%time

param_grid = {'max_iter': [100, 1000, 2000, 3000, 5000, 10],
              'penalty' : ['l1', 'l2', 'elasticnet'],
              'class_weight' : ['balanced'],
              'multi_class': ['ovr'],

}
lg = LogisticRegression()
grid = GridSearchCV(lg, param_grid, cv=10, scoring='accuracy')

grid.fit(X_train,y_train)

CPU times: user 9.45 s, sys: 62 ms, total: 9.51 s
Wall time: 9.61 s


GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'class_weight': ['balanced'],
                         'max_iter': [100, 1000, 2000, 3000, 5000, 10],
                         'multi_class': ['ovr'],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='accuracy')

In [ ]:
print(grid.best_score_)
print(grid.best_estimator_)

0.9153655641288513
LogisticRegression(class_weight='balanced', multi_class='ovr')


In [ ]:
model = LogisticRegression(class_weight='balanced', multi_class='ovr')
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93      1038
           1       0.95      0.91      0.93      1081

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



### 2-й способ (sklearn)

In [ ]:
# с использованием word2vec

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

  0%|          | 0/6420 [00:00<?, ?it/s]

In [ ]:
sentences[0]

['cdc',
 'currently',
 'report',
 'death',
 'general',
 'discrepancy',
 'death',
 'count',
 'different',
 'source',
 'small',
 'explicable',
 'death',
 'toll',
 'stand',
 'roughly',
 'people',
 'today']

In [ ]:
%time model_tweets = Word2Vec(sentences, workers=4, vector_size=300, min_count=3, window=5)

CPU times: user 1.47 s, sys: 28.6 ms, total: 1.5 s
Wall time: 919 ms


In [ ]:
len(model_tweets.wv['currently'])

300

In [ ]:
model_tweets.wv.get_vector('currently')

array([-2.20664544e-04,  2.87634224e-01, -2.72945743e-02,  1.18417829e-01,
        6.20076805e-02, -3.30079883e-01,  1.54702723e-01,  5.93825519e-01,
        4.07370590e-02, -1.33193329e-01,  6.21239329e-03, -1.79142058e-01,
        2.88877375e-02, -8.40579122e-02, -2.74988562e-01, -2.36834854e-01,
        1.22486874e-01, -8.84705186e-02, -3.42789479e-02, -6.94365874e-02,
       -1.54546380e-01, -5.47732450e-02,  2.39937201e-01,  1.72606148e-02,
        2.56611735e-01, -2.29593553e-02, -3.84063900e-01,  6.15317598e-02,
       -2.06407502e-01, -2.88634151e-01,  5.76631092e-02, -1.29115775e-01,
        6.73372671e-03, -1.45618515e-02, -9.92831066e-02,  8.42448175e-02,
        1.31444886e-01, -3.10262144e-01, -6.02473691e-02,  3.67598906e-02,
       -1.88179284e-01,  1.30343167e-02,  3.23995836e-02, -2.61484593e-01,
        1.43123731e-01,  1.31757751e-01,  1.01722695e-01,  1.29527360e-01,
       -4.28594537e-02,  2.52932370e-01,  8.84322673e-02,  5.05620092e-02,
       -9.53716487e-02,  

In [ ]:
# создание эмбеддингов
def get_text_embedding_1(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.average(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [ ]:
features_1 = [get_text_embedding_1(text) for text in tqdm(df.tweet)]

  0%|          | 0/6420 [00:00<?, ?it/s]

In [ ]:
len(features_1)

6420

In [ ]:
len(features_1[0])

300

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)

In [ ]:
%%time

param_grid = {'max_iter': [1000, 2000, 3000, 5000, 10000],
              'epsilon' : [0.1, 0.2, 0.5, 0.01],
              'learning_rate' : ['constant', 'optimal', 'invscaling', 'adaptive']

}
sgd = SGDClassifier()
grid = GridSearchCV(sgd, param_grid, cv=10, scoring='accuracy')

grid.fit(X_train,y_train)

CPU times: user 32.5 s, sys: 57.1 ms, total: 32.5 s
Wall time: 32.6 s


GridSearchCV(cv=10, estimator=SGDClassifier(),
             param_grid={'epsilon': [0.1, 0.2, 0.5, 0.01],
                         'learning_rate': ['constant', 'optimal', 'invscaling',
                                           'adaptive'],
                         'max_iter': [1000, 2000, 3000, 5000, 10000]},
             scoring='accuracy')

In [ ]:
print(grid.best_score_)
print(grid.best_estimator_)

0.9028138995305671
SGDClassifier(max_iter=10000)


In [ ]:
model = SGDClassifier(max_iter=10000, epsilon=0.2, random_state=42)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.88      0.90      1006
        real       0.90      0.93      0.92      1113

    accuracy                           0.91      2119
   macro avg       0.91      0.91      0.91      2119
weighted avg       0.91      0.91      0.91      2119



### 3-й способ (sklearn)

TruncatedSVD


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD

In [ ]:
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('tm', TruncatedSVD()),
    ('clf', RandomForestClassifier())
])


In [ ]:
X = df.tweet

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, df.label, test_size=0.33)

In [ ]:
clf_tf_idf_TruncatedSVD = Pipeline(
    [('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('tsvd', TruncatedSVD()),
    #  ('clf', SGDClassifier(max_iter=10000, epsilon=0.2, random_state=42))
     ('clf', RandomForestClassifier())

     ]
)

In [ ]:
clf_tf_idf_TruncatedSVD.fit(X_train, y_train)

pred = clf_tf_idf_TruncatedSVD.predict(X_test)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

        fake       0.62      0.73      0.67       897
        real       0.77      0.67      0.72      1222

    accuracy                           0.69      2119
   macro avg       0.69      0.70      0.69      2119
weighted avg       0.71      0.69      0.70      2119



In [ ]:
clf_tf_idf_LDA = Pipeline(
    [('vect', CountVectorizer()),
     ('lda', LatentDirichletAllocation()),
     ('clf', LogisticRegression())]
)

params_tf_idf_LDA={
    'vect__analyzer': ['word'],
    'vect__max_df': [0.75],
    'vect__ngram_range': [(1, 1), (2, 2), (3, 3)],
    'lda__n_components' : [25, 50, 100],
    'clf__C': np.logspace(-3, 3, 7),
    # 'clf__penalty': ['l1']
}

scores=['accuracy', 'f1']


In [ ]:
grid_tf_idf_LDA = GridSearchCV(
    clf_tf_idf_LDA,
    param_grid=params_tf_idf_LDA,
    cv=10,
    scoring=scores,
    refit=scores[0],
    n_jobs=-1,
    verbose=1
)

In [ ]:
%%time
grid_tf_idf_LDA.fit(X_train, y_train)

Fitting 10 folds for each of 63 candidates, totalling 630 fits
CPU times: user 58.4 s, sys: 9.09 s, total: 1min 7s
Wall time: 2h 4min 7s


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('lda', LatentDirichletAllocation()),
                                       ('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'clf__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'lda__n_components': [25, 50, 100],
                         'vect__analyzer': ['word'], 'vect__max_df': [0.75],
                         'vect__ngram_range': [(1, 1), (2, 2), (3, 3)]},
             refit='accuracy', scoring=['accuracy', 'f1'], verbose=1)

In [ ]:
print(grid_tf_idf_LDA.best_score_)
print(grid_tf_idf_LDA.best_estimator_)

0.813061026277451
Pipeline(steps=[('vect', CountVectorizer(max_df=0.75)),
                ('lda', LatentDirichletAllocation(n_components=100)),
                ('clf', LogisticRegression(C=100.0))])


In [ ]:
clf_tf_idf_LDA = Pipeline(
    [('vect', CountVectorizer(max_df=0.75)),
     ('lda', LatentDirichletAllocation(n_components=100)),
     ('clf', LogisticRegression(C=100.0))]
)

In [ ]:
clf_tf_idf_LDA.fit(X_train, y_train)

pred = clf_tf_idf_LDA.predict(X_test)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

        fake       0.75      0.82      0.78       965
        real       0.84      0.77      0.80      1154

    accuracy                           0.79      2119
   macro avg       0.79      0.79      0.79      2119
weighted avg       0.80      0.79      0.79      2119



### Метод на pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

In [ ]:
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [ ]:
len(token_lists[0])

18

In [ ]:
features = [get_word_embedding(text, 100) for text in tqdm(token_lists)]
# мы получили массив из векторов длиной в 100 (каждый элемент - это слово переведенное в w2v-вектор длиной 300)

  0%|          | 0/6420 [00:00<?, ?it/s]

In [ ]:
features[0][0].shape

(300,)

In [ ]:
features[0] # список длиной 100 из векторов w2v

In [ ]:
len(features[0])

100

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 300 - длина эмбединга слова, 100 длина вектора одного элемента (признака, или предложения)
        self.lstm = nn.LSTM(300, 100)
        self.lstm = nn.LSTM(300, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))

        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
net.to(device)
print(net)


Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.25)

In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
in_data.shape

torch.Size([4815, 100, 300])

In [ ]:
targets.shape

torch.Size([4815])

In [ ]:
targets

tensor([0., 1., 0.,  ..., 0., 0., 1.])

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=32):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size].to(device)
        batch_y = targets[i:i + batch_size].to(device)
        # print('batch_x', batch_x.shape)
        # print('batch_y', batch_y.shape)

        optimizer.zero_grad()
        output = net(batch_x)
        # print('output', output.shape)

        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
net.train()

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)

In [ ]:
for i in range(2):
  train_one_epoch(in_data, targets)

  0%|          | 0/151 [00:00<?, ?it/s]

tensor(0.6931, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


  0%|          | 0/151 [00:00<?, ?it/s]

tensor(0.6931, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [ ]:
# Проверка результата
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

with torch.no_grad():
    output = net(in_data_test.cuda()).reshape(-1)

result = (output.cpu() > 0.5) == targets_test

result.sum().item() / len(result)

0.47912772585669783